# Optimize your Docker Infrastructure with Python

## PyData NYC

November 11, 2015

[Ryan J. O'Neil](mailto:ryanjoneil@gmail.com)  
Lead Engineer @ [Yhat, Inc.](https://yhathq.com/)  
[SEOR](http://seor.gmu.edu) PhD Candidate @ George Mason University

### Motivation

Consider the devops engineer.

A lot of devops revolves around the creating environments for:

* Software development
* Testing & quality assurance
* Running operational systems

There's even a Venn diagram about this...  

![](images/devops-venn.svg)

A simple example is the upgrading of _My Enterprise Java Application (TM)_ to a new database. _My Enterprise Java Application (TM)_ generates lots of \$\$\$\$ for _My Enterprise Software Company_ and it keeps me gainfully employed!

Say an underlying system utility we're using is updated with major performance improvements.

I don't want to just upgrade the utility on our production instance without testing first. So I need two environments from my devops person.

One for running the production instance:

|     | Command                                                        |
|-----|----------------------------------------------------------------|
| $A$ | Install the Java compiler and runtime environment.             |
| $B$ | Download a set of external dependencies.                       |
| $C$ | Set up a an EntepriseDB (TM) schema and populate it with data. |

And another for testing it with the new version of the utility:

|     | Command                                                        |
|-----|----------------------------------------------------------------|
| $A$ | Install the Java compiler and runtime environment.             |
| $B$ | Download a set of external dependencies.                       |
| $C$ | Set up a an EntepriseDB (TM) schema and populate it with data. |
| $D$ | Update the underlying system utility.                          |

If _My Enterprise Java Application (TM)_ integrates with a number of optional third party applications, I might need an additional test environment for each one.

![](images/saturday.png)

At a big software shop, devops may be responsible for the continual setup and teardown of dozens (or _hundreds_) of system configurations for a single software project.

This used to be done with physical hardware on _(sometimes)_ fresh operating system installs.

Most medium-to-large software shops had an air conditioned room that looked like this.

![](images/cable-spaghetti.jpg)

### Background

#### Docker Cache Mechanics

### Problem Formulation